In [157]:
import ijson
import json
import numpy as np
import pandas as pd
import os
from pandas.io.json import json_normalize

In [158]:
azure_json_path = './inputs/azure_FRIENDS.json'

## Celebrities:

In [159]:
with open(azure_json_path, 'r') as f:
    celebrity_tracks_list = list(ijson.items(f, 'videos.item.insights.faces.item'))
df = json_normalize(celebrity_tracks_list)
df

,id,name,confidence,description,thumbnailId,referenceId,referenceType,title,imageUrl,thumbnails,instances
0,1279,David Schwimmer,1,David Lawrence Schwimmer is an American actor ...,fc381241-ca1a-4195-bae5-88e9c3e0ce30,e51c3e8e-93a6-fb48-fb5c-d4af60942446,Bing,American Actor,https://www.bing.com/th?id=AMMS_2400447960bb27...,[{'id': '1c68f54b-a4a3-4e4e-96d5-0bcacff22124'...,[{'thumbnailsIds': ['1c68f54b-a4a3-4e4e-96d5-0...
1,1007,Courteney Cox,0.9986,"Courteney Bass Cox is an American actress, pro...",a3b6cabc-3968-4e8a-9bd0-342502dfb6e4,34196a16-1477-c022-e099-135831a05061,Bing,American Actress,https://www.bing.com/th?id=AMMS_bc02ff04fe1dee...,[{'id': '6d89b578-1317-4743-bfb8-49ac34df9c52'...,[{'thumbnailsIds': ['6d89b578-1317-4743-bfb8-4...
2,1225,Matt LeBlanc,1,Matthew Steven LeBlanc is an American actor an...,11d81d2d-1312-4b7d-bb3b-4289da908f4a,cf39bd07-bedf-0457-7106-0c6d60107d13,Bing,American Actor,https://www.bing.com/th?id=AMMS_b4adeefb0ac306...,[{'id': 'a4de48ee-d260-4502-9413-877d107688a3'...,[{'thumbnailsIds': ['a4de48ee-d260-4502-9413-8...
3,1011,Lisa Kudrow,0.9929,"Lisa Valerie Kudrow is an American actress, co...",46d43daa-9753-4a3a-86e9-7256de52ec56,b3936f96-4b79-5248-3e71-6f01c4134d5b,Bing,American Actress,https://www.bing.com/th?id=AMMS_9d45e5a299dec4...,[{'id': '46d43daa-9753-4a3a-86e9-7256de52ec56'...,[{'thumbnailsIds': ['46d43daa-9753-4a3a-86e9-7...
4,1001,Jennifer Aniston,0.9677,Jennifer Joanna Aniston is an American actress...,3ec35e8e-1fa2-44ab-92b0-8d8ac7de3f6a,0ab0dc3f-f999-06b4-c19a-5e50b745cec4,Bing,American Actress,https://www.bing.com/th?id=AMMS_49e8b448e95989...,[{'id': 'ed4b6dd8-25d9-42d8-b127-f99493c56a85'...,[{'thumbnailsIds': ['ed4b6dd8-25d9-42d8-b127-f...
5,1253,Matthew Perry,0.8623,Matthew Langford Perry is a Canadian-American ...,0176ebf6-0dba-4110-b1de-95f8800cab4d,a58d749d-71ff-6ab2-8bd0-3b2b6ad3bdee,Bing,Canadian-American Actor,https://www.bing.com/th?id=AMMS_623f84b3be9131...,[{'id': '722e4df8-3da4-4e86-9690-419a5164776e'...,[{'thumbnailsIds': ['722e4df8-3da4-4e86-9690-4...
6,1103,Unknown #1,0,None,87e6c37a-7fb6-4311-84d8-1d0fce0976e2,NaN,NaN,None,None,[{'id': '87e6c37a-7fb6-4311-84d8-1d0fce0976e2'...,"[{'adjustedStart': '0:00:01.251', 'adjustedEnd..."
7,1177,Unknown #2,0,None,073a08ff-5cfa-445b-88bc-e938b3153de3,NaN,NaN,None,None,[{'id': '073a08ff-5cfa-445b-88bc-e938b3153de3'...,[{'thumbnailsIds': ['073a08ff-5cfa-445b-88bc-e...
8,1304,Unknown #3,0,None,af37af7f-0f9b-47d2-90e7-fee06984dbb7,NaN,NaN,None,None,[{'id': 'af37af7f-0f9b-47d2-90e7-fee06984dbb7'...,[{'thumbnailsIds': ['af37af7f-0f9b-47d2-90e7-f...
9,1084,Unknown #4,0,None,5bfe1f85-73f6-49b8-befc-13f62850d3a2,NaN,NaN,None,None,[{'id': '5bfe1f85-73f6-49b8-befc-13f62850d3a2'...,[{'thumbnailsIds': ['2f086e08-3cb4-4594-b697-1...


In [160]:
df['source'] = 'microsoft'
segment_df = df.drop(['description', 'thumbnailId', 'referenceId', 'referenceType', 'title', 'thumbnails'], axis=1)\
               .rename(columns = {"imageUrl":"url"})
segment_df = segment_df.explode('instances').reset_index(drop=True)
segment_df['instances'].iloc[0]

{'thumbnailsIds': ['1c68f54b-a4a3-4e4e-96d5-0bcacff22124'],
 'adjustedStart': '0:00:09.259',
 'adjustedEnd': '0:00:10.177',
 'start': '0:00:09.259',
 'end': '0:00:10.177'}

In [161]:
def convert_to_secs(time_str):
    h, m, s = time_str.split(':')
    return int(h)*3600 + int(m)*60 + float(s)

In [162]:
segment_df['segment.start_time'] = segment_df['instances'].apply(lambda x: convert_to_secs(x['start']) if pd.notna(x) else x)
segment_df['segment.end_time'] = segment_df['instances'].apply(lambda x: convert_to_secs(x['end']) if pd.notna(x) else x)
segment_df.drop('instances', axis=1, inplace=True)
segment_df

,id,name,confidence,url,source,segment.start_time,segment.end_time
0,1279,David Schwimmer,1,https://www.bing.com/th?id=AMMS_2400447960bb27...,microsoft,9.259,10.177
1,1279,David Schwimmer,1,https://www.bing.com/th?id=AMMS_2400447960bb27...,microsoft,18.018,20.896
2,1279,David Schwimmer,1,https://www.bing.com/th?id=AMMS_2400447960bb27...,microsoft,23.023,23.983
3,1007,Courteney Cox,0.9986,https://www.bing.com/th?id=AMMS_bc02ff04fe1dee...,microsoft,0.000,1.252
4,1007,Courteney Cox,0.9986,https://www.bing.com/th?id=AMMS_bc02ff04fe1dee...,microsoft,3.337,3.671
5,1007,Courteney Cox,0.9986,https://www.bing.com/th?id=AMMS_bc02ff04fe1dee...,microsoft,18.018,20.896
6,1225,Matt LeBlanc,1,https://www.bing.com/th?id=AMMS_b4adeefb0ac306...,microsoft,0.000,1.252
7,1225,Matt LeBlanc,1,https://www.bing.com/th?id=AMMS_b4adeefb0ac306...,microsoft,11.595,12.471
8,1225,Matt LeBlanc,1,https://www.bing.com/th?id=AMMS_b4adeefb0ac306...,microsoft,13.764,14.932
9,1225,Matt LeBlanc,1,https://www.bing.com/th?id=AMMS_b4adeefb0ac306...,microsoft,26.777,27.528


## Logos:
Output from 'brands' endpoint is not present.


## Labels:

In [163]:
with open(azure_json_path, 'r') as f:
    labels_list = list(ijson.items(f, 'videos.item.insights.labels.item'))
df = json_normalize(labels_list)
df

,id,name,referenceId,language,instances
0,1,People,person,en-US,"[{'confidence': 0.9965, 'adjustedStart': '0:00..."
1,2,Indoor,NaN,en-US,"[{'confidence': 0.9422, 'adjustedStart': '0:00..."
2,3,Outdoor,NaN,en-US,"[{'confidence': 0.9243, 'adjustedStart': '0:00..."
3,4,Wall,structure/wall,en-US,"[{'confidence': 0.99, 'adjustedStart': '0:00:1..."
4,5,Standing,NaN,en-US,"[{'confidence': 0.89, 'adjustedStart': '0:00:1..."
5,6,Man,person/man,en-US,"[{'confidence': 0.9717, 'adjustedStart': '0:00..."
6,7,Kitchen,structure/room/kitchen,en-US,"[{'confidence': 0.8787, 'adjustedStart': '0:00..."
7,8,Sitting,action/sitting,en-US,"[{'confidence': 0.9916, 'adjustedStart': '0:00..."
8,9,People,NaN,en-US,"[{'confidence': 0.8831, 'adjustedStart': '0:00..."
9,10,Ceiling,surface/ceiling,en-US,"[{'confidence': 0.8009, 'adjustedStart': '0:00..."


In [164]:
df['source'] = 'microsoft'
df = df.explode('instances').reset_index(drop=True)
df = df.join(json_normalize(df['instances']))
df['segment.start_time'] = df['start'].apply(lambda x: convert_to_secs(x) if pd.notna(x) else x)
df['segment.end_time'] = df['end'].apply(lambda x: convert_to_secs(x) if pd.notna(x) else x)
df.drop(['instances', 'adjustedStart', 'adjustedEnd', 'start', 'end'], axis=1, inplace=True)
df

,id,name,referenceId,language,source,confidence,segment.start_time,segment.end_time
0,1,People,person,en-US,microsoft,0.9965,1.335,13.347
1,1,People,person,en-US,microsoft,0.998,14.681,28.028
2,2,Indoor,NaN,en-US,microsoft,0.9422,1.335,6.674
3,2,Indoor,NaN,en-US,microsoft,0.9323,8.008,9.343
4,2,Indoor,NaN,en-US,microsoft,0.963,10.677,28.028
5,3,Outdoor,NaN,en-US,microsoft,0.9243,6.673,8.008
6,3,Outdoor,NaN,en-US,microsoft,0.9881,9.343,10.678
7,4,Wall,structure/wall,en-US,microsoft,0.99,10.677,12.012
8,4,Wall,structure/wall,en-US,microsoft,0.9566,24.024,25.359
9,5,Standing,NaN,en-US,microsoft,0.89,10.677,13.347


## Text/OCR:

In [165]:
with open(azure_json_path, 'r') as f:
    ocr_list = list(ijson.items(f, 'videos.item.insights.ocr.item'))
df = json_normalize(ocr_list)
df

,id,text,confidence,left,top,width,height,language,instances
0,1,"""Friends""",0.9174,50,28,80,18,en-US,"[{'adjustedStart': '0:00:11.929', 'adjustedEnd..."
1,2,Now being distributed on Hu,0.9324,51,50,119,16,en-US,"[{'adjustedStart': '0:00:11.929', 'adjustedEnd..."
2,3,"& CWYo"" ""Day"" 0 Ente ""n nc-",0.5238,62,74,194,9,en-US,"[{'adjustedStart': '0:00:13.764', 'adjustedEnd..."
3,4,"& eWyo ""Sun ro Ente Le i I i nc-",0.4822,62,74,194,9,en-US,"[{'adjustedStart': '0:00:14.223', 'adjustedEnd..."
4,5,"& C'W'n ""Day""os.Sun 0"" Thing 1 Nc",0.4651,62,74,191,9,en-US,"[{'adjustedStart': '0:00:15.14', 'adjustedEnd'..."
5,6,"& CW'0 ""Day"" os.Day 0"" 1",0.4397,62,74,154,9,en-US,"[{'adjustedStart': '0:00:15.599', 'adjustedEnd..."
6,7,"& CW ""Day 0 0"" Thing n Inc",0.4702,62,74,191,9,en-US,"[{'adjustedStart': '0:00:18.352', 'adjustedEnd..."
7,8,"& W ""n"" ""0 days 0"" thing n i nc.",0.4714,62,74,194,9,en-US,"[{'adjustedStart': '0:00:18.81', 'adjustedEnd'..."
8,9,"& CW ""Day"" os. Ente ""Thingn n nc.",0.4848,62,74,191,9,en-US,"[{'adjustedStart': '0:00:21.104', 'adjustedEnd..."
9,10,I nc.,0.5448,230,74,26,9,en-US,"[{'adjustedStart': '0:00:16.517', 'adjustedEnd..."


In [166]:
df['source'] = 'microsoft'
df = df.explode('instances').reset_index(drop=True)
df['segment.start_time'] = df['instances'].apply(lambda x: convert_to_secs(x['start']) if pd.notna(x) else x)
df['segment.end_time'] = df['instances'].apply(lambda x: convert_to_secs(x['end']) if pd.notna(x) else x)


In [167]:
def add_bounding_box(row):
    temp_dict = { 'left': row['left'], \
                  'right': row['left'] + row['width'], \
                  'top' : row['top'], \
                  'bottom' : row['top'] + row['height'] }
    return temp_dict

In [168]:
df['bounding_box'] = df.apply(add_bounding_box, axis=1)
df.drop(['instances', 'left', 'top', 'width', 'height'], axis=1, inplace=True)
df

,id,text,confidence,language,source,segment.start_time,segment.end_time,bounding_box
0,1,"""Friends""",0.9174,en-US,microsoft,11.929,16.058,"{'left': 50, 'right': 130, 'top': 28, 'bottom'..."
1,1,"""Friends""",0.9174,en-US,microsoft,16.517,16.976,"{'left': 50, 'right': 130, 'top': 28, 'bottom'..."
2,1,"""Friends""",0.9174,en-US,microsoft,17.434,17.893,"{'left': 50, 'right': 130, 'top': 28, 'bottom'..."
3,1,"""Friends""",0.9174,en-US,microsoft,18.352,19.269,"{'left': 50, 'right': 130, 'top': 28, 'bottom'..."
4,1,"""Friends""",0.9174,en-US,microsoft,21.104,22.940,"{'left': 50, 'right': 130, 'top': 28, 'bottom'..."
5,2,Now being distributed on Hu,0.9324,en-US,microsoft,11.929,16.058,"{'left': 51, 'right': 170, 'top': 50, 'bottom'..."
6,2,Now being distributed on Hu,0.9324,en-US,microsoft,18.352,19.269,"{'left': 51, 'right': 170, 'top': 50, 'bottom'..."
7,2,Now being distributed on Hu,0.9324,en-US,microsoft,21.104,22.940,"{'left': 51, 'right': 170, 'top': 50, 'bottom'..."
8,3,"& CWYo"" ""Day"" 0 Ente ""n nc-",0.5238,en-US,microsoft,13.764,14.223,"{'left': 62, 'right': 256, 'top': 74, 'bottom'..."
9,4,"& eWyo ""Sun ro Ente Le i I i nc-",0.4822,en-US,microsoft,14.223,15.140,"{'left': 62, 'right': 256, 'top': 74, 'bottom'..."


## Speech:

In [169]:
with open(azure_json_path, 'r') as f:
    transcript_list = list(ijson.items(f, 'videos.item.insights.transcript.item'))
df = json_normalize(transcript_list)
df

,id,text,confidence,speakerId,language,instances
0,1,Right.,1,1,en-US,"[{'adjustedStart': '0:00:00.57', 'adjustedEnd'..."
1,2,SNS This is Tsukuba Iceland.,1,2,en-US,"[{'adjustedStart': '0:00:05.38', 'adjustedEnd'..."


In [170]:
df['source'] = 'microsoft'
df = df.explode('instances').reset_index(drop=True)
df['segment.start_time'] = df['instances'].apply(lambda x: convert_to_secs(x['start']) if pd.notna(x) else x)
df['segment.end_time'] = df['instances'].apply(lambda x: convert_to_secs(x['end']) if pd.notna(x) else x)
df.drop(['instances', 'speakerId'], axis=1, inplace=True)
df

,id,text,confidence,language,source,segment.start_time,segment.end_time
0,1,Right.,1,en-US,microsoft,0.57,4.629
1,2,SNS This is Tsukuba Iceland.,1,en-US,microsoft,5.38,30.030


## Shots:

In [171]:
with open(azure_json_path, 'r') as f:
    shot_list = list(ijson.items(f, 'videos.item.insights.shots.item'))
df = json_normalize(shot_list)
df

,id,tags,keyFrames,instances
0,1,"[Wide, Indoor]","[{'id': 1, 'instances': [{'thumbnailId': '9e8d...","[{'adjustedStart': '0:00:00', 'adjustedEnd': '..."
1,2,"[Wide, CenterFace, Indoor, Medium]","[{'id': 3, 'instances': [{'thumbnailId': 'e3b2...","[{'adjustedStart': '0:00:02.753', 'adjustedEnd..."
2,3,"[Indoor, Medium]","[{'id': 5, 'instances': [{'thumbnailId': '6000...","[{'adjustedStart': '0:00:05.714', 'adjustedEnd..."
3,4,"[Medium, RightFace, Outdoor]","[{'id': 7, 'instances': [{'thumbnailId': '2e57...","[{'adjustedStart': '0:00:06.84', 'adjustedEnd'..."
4,5,[Indoor],"[{'id': 9, 'instances': [{'thumbnailId': 'aed7...","[{'adjustedStart': '0:00:08.926', 'adjustedEnd..."
5,6,"[Medium, CenterFace, Outdoor]","[{'id': 10, 'instances': [{'thumbnailId': '143...","[{'adjustedStart': '0:00:10.219', 'adjustedEnd..."
6,7,"[Wide, Indoor]","[{'id': 11, 'instances': [{'thumbnailId': '8c6...","[{'adjustedStart': '0:00:11.887', 'adjustedEnd..."
7,8,"[Indoor, Medium, CenterFace]","[{'id': 13, 'instances': [{'thumbnailId': '569...","[{'adjustedStart': '0:00:13.639', 'adjustedEnd..."
8,9,"[Medium, CenterFace, Indoor, Wide]","[{'id': 15, 'instances': [{'thumbnailId': 'de1...","[{'adjustedStart': '0:00:14.932', 'adjustedEnd..."
9,10,"[Medium, Indoor]","[{'id': 17, 'instances': [{'thumbnailId': 'a42...","[{'adjustedStart': '0:00:18.06', 'adjustedEnd'..."


In [172]:
df['source'] = 'microsoft'
df = df.explode('instances').reset_index(drop=True)
df['segment.start_time'] = df['instances'].apply(lambda x: convert_to_secs(x['start']) if pd.notna(x) else x)
df['segment.end_time'] = df['instances'].apply(lambda x: convert_to_secs(x['end']) if pd.notna(x) else x)

In [173]:
def handle_keyframes(input_list):
    out_list = []
    for keyframe in input_list:
        instances = keyframe.pop('instances', None)
        if instances:
            keyframe['start_time'] = convert_to_secs(instances[0]['start'])
            keyframe['end_time'] = convert_to_secs(instances[0]['end'])
    return input_list

In [174]:
df['keyframes'] = df['keyFrames'].apply(handle_keyframes)
df.drop(['instances', 'keyFrames'], axis=1, inplace=True)
df

,id,tags,source,segment.start_time,segment.end_time,keyframes
0,1,"[Wide, Indoor]",microsoft,0.000,2.753,"[{'id': 1, 'start_time': 0.25, 'end_time': 0.2..."
1,2,"[Wide, CenterFace, Indoor, Medium]",microsoft,2.753,5.714,"[{'id': 3, 'start_time': 2.753, 'end_time': 2...."
2,3,"[Indoor, Medium]",microsoft,5.714,6.840,"[{'id': 5, 'start_time': 5.714, 'end_time': 5...."
3,4,"[Medium, RightFace, Outdoor]",microsoft,6.840,8.926,"[{'id': 7, 'start_time': 6.84, 'end_time': 6.8..."
4,5,[Indoor],microsoft,8.926,10.219,"[{'id': 9, 'start_time': 8.926, 'end_time': 8...."
5,6,"[Medium, CenterFace, Outdoor]",microsoft,10.219,11.887,"[{'id': 10, 'start_time': 10.219, 'end_time': ..."
6,7,"[Wide, Indoor]",microsoft,11.887,13.639,"[{'id': 11, 'start_time': 11.887, 'end_time': ..."
7,8,"[Indoor, Medium, CenterFace]",microsoft,13.639,14.932,"[{'id': 13, 'start_time': 13.639, 'end_time': ..."
8,9,"[Medium, CenterFace, Indoor, Wide]",microsoft,14.932,18.060,"[{'id': 15, 'start_time': 14.932, 'end_time': ..."
9,10,"[Medium, Indoor]",microsoft,18.060,20.938,"[{'id': 17, 'start_time': 18.06, 'end_time': 1..."


## Content Moderation:

In [175]:
with open(azure_json_path, 'r') as f:
    moderation_list = list(ijson.items(f, 'videos.item.insights.visualContentModeration.item'))
df = json_normalize(moderation_list)
df

,id,adultScore,racyScore,instances
0,1,0.0024,0.9296,"[{'adjustedStart': '0:00:13.931', 'adjustedEnd..."


In [176]:
df['source'] = 'microsoft'
df = df.explode('instances').reset_index(drop=True)
df['segment.start_time'] = df['instances'].apply(lambda x: convert_to_secs(x['start']) if pd.notna(x) else x)
df['segment.end_time'] = df['instances'].apply(lambda x: convert_to_secs(x['end']) if pd.notna(x) else x)
df.drop('instances', axis=1, inplace=True)
df

,id,adultScore,racyScore,source,segment.start_time,segment.end_time
0,1,0.0024,0.9296,microsoft,13.931,13.973


**Currently not using textualContentModeration, as there are no samples.**

In [177]:
with open(azure_json_path, 'r') as f:
    moderation_list = list(ijson.items(f, 'videos.item.insights.textualContentModeration'))
df = json_normalize(moderation_list)
df

,id,bannedWordsCount,bannedWordsRatio,instances
0,0,0,0,[]
